In [1]:
import re
import requests
from bs4 import BeautifulSoup as bs
from db import FileDB

# DB 정의
file_db = FileDB()

In [2]:
# 상수 정의

# 세부적인 카테고리를 대표 카테고리로 통일하기 위한 map
CATEGORY_MAP = {
        # 취업
        "강소기업채용" : "취업",
        "채용정보" : "취업",
        "인턴쉽" : "취업",
        "교육프로그램" : "취업",
        "고시반" : "취업",
        "기타" : "취업",

        # 장학
        "국가장학금" : "장학",
        "교외장학금" : "장학",
        "교내장학금" : "장학",
        "면학근로" : "장학",
        "학자금대출" : "장학",
        "국가근로" : "장학",
        "공모전 등" : "장학",
        "비교과장학" : "장학",

        # 창업
        "창업정보" : "창업",
        "창업공모전" : "창업",
        "창업행사" : "창업",
        "기타" : "창업",
    }
# RSS 피드의 페이지 번호를 지정하기 위한 기본 URL. {} 부분에 숫자가 들어감.
BASE_URL = 'https://www.hansung.ac.kr/bbs/hansung/143/rssList.do?page={}'
# 상대 경로를 절대 경로로 변환할 때 사용할 기본 도메인
BASE_DOMAIN = "https://www.hansung.ac.kr"

In [3]:
# 함수 정의

# 카테고리 정규화 함수
def normalize_category(category):
    return CATEGORY_MAP.get(category, category)

# 공지사항 게시글을 조회하여 내용, 사진, 첨부파일 수집하는 함수
def html_croll(url, base_domain):
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')

    content, image_urls, attachments = "", [], []
    
    # 내용 및 사진
    view_con_div = soup.find('div', class_='view-con')
    if view_con_div:
        content = view_con_div.get_text(strip=True) # 본문 텍스트 전체 가져오기
        # 그 안의 모든 텍스트를 태그 제거 + 붙여서 가져옴
        # 표(table)나 문단(p) 같은 게 전부 줄바꿈 없이 한 덩어리로 합쳐져 버림
        
        # 이미지 URL을 찾기 (content와 상관없이 이미지 URL을 추출)
        for img_tag in view_con_div.find_all('img', src=True):
            src = img_tag['src']
            if src.startswith('/'):
                src = f"{base_domain}{src}" # 기본 도메인 주소 사용
            image_urls.append(src)
    else:
        content = "No content found"

    # 첨부파일
    file_div = soup.find('div', class_='view-file')
    if file_div:
        for a_tag in file_div.find_all('a', href=True):
            href = a_tag['href']
            # 다운로드 링크만 걸러내기
            if "download.do" in href:
                if href.startswith('/'):
                    file_url = f"{base_domain}{href}"
                file_name = a_tag.get_text(strip=True)
                attachments.append(f"{file_name} | {file_url}") # 문자열로 합쳐서 저장
    
    return content, image_urls, attachments

# RSS 피드를 순회하여 제목, 링크, 게시일, 카테고리 수집하는 함수
def rss_croll(db, max_pages, base_url, base_domain):
    saved_cnt = 0 # 저장된 공지사항 개수
    image_only_count = 0 # 이미지만 있는 공지사항 개수

    for page_number in range(1, max_pages+1):
        url = base_url.format(page_number)
        page = requests.get(url)
        soup = bs(page.text, 'xml')
        items = soup.find_all('item')

        if not items:
            print(f"{page_number} 페이지에 더 이상 게시물이 없습니다.")
            break # 게시물이 없으면 중단

        for item in items:
            title = item.find('title').get_text(strip=True) if item.find('title') else "No Title"
            link = item.find('link').get_text() if item.find('link') else "No Link"
            pub_date = item.find('pubDate').get_text(strip=True) if item.find('pubDate') else "No Date"
            category = item.find('category').get_text(strip=True) if item.find('category') else "No Category"

            # 카테고리 정규화
            category = normalize_category(category)

            # 절대 경로로 변경
            if link.startswith("/"):
                link = f"{base_domain}{link}"

            # 내용, 사진, 첨부파일들
            content, image_urls, attachments = html_croll(link, base_domain)

            # 내용은 없고, 이미지 URL은 있는지 확인
            if image_urls and content == "":
                image_only_count += 1
                print(f"-> 이미지 전용 공지 발견: {title}")

            # '143/' 뒤에 오는 숫자 그룹을 찾는 정규표현식
            match = re.search(r'143/(\d+)', link)
            notice_id = match.group(1)

            # DB에 저장
            db.save_notice(
                notice_id=notice_id,
                title=title,
                link=link,
                date=pub_date,
                category=category,
                content=content,
                image_urls=image_urls,
                attachments=attachments
            )
            saved_cnt += 1
            
    print(f"총 {saved_cnt}개의 공지사항이 성공적으로 저장되었습니다!")
    print(f"이미지만 있는 공지는 총 {image_only_count}개입니다.")

In [6]:
# 실행

page = 2

rss_croll(
    db=file_db,
    max_pages=page,
    base_url=BASE_URL,
    base_domain=BASE_DOMAIN
)

-> 이미지 전용 공지 발견: 2026학년도 1학기 일어권 파견 교환학생 선발안내
-> 이미지 전용 공지 발견: 2025년 세종시립청소년교향악단 예능단원 모집 안내
-> 이미지 전용 공지 발견: 2025년 상반기분 통영시 대학생 학자금 이자 지원 신청 안내(9/22 마감)
-> 이미지 전용 공지 발견: 2025년도 하반기 (재)달서인재육성장학재단 장학생 선발 안내(9/30 마감)
-> 이미지 전용 공지 발견: 케이키친한상 카페 시식 및 운영 안내
-> 이미지 전용 공지 발견: 2025년「도봉구 청년 사회첫출발 지원금」홍보
-> 이미지 전용 공지 발견: 2025년도 (재)포항시장학회 귀뚜라미 장학생 선발 안내(9/18 마감)
-> 이미지 전용 공지 발견: 2025년도 (재)포항시장학회 대학교 장학생 선발 안내(9/18 마감)
-> 이미지 전용 공지 발견: 25년도 2학기 중소기업 취업연계 장학사업(희망사다리 1유형) 신규장학생 신청 안내 (~9/19 마감)
-> 이미지 전용 공지 발견: 2025년도 익산사랑장학재단 장학생 선발 안내(9/10 마감)
-> 이미지 전용 공지 발견: [2025학년도 2학기] 동아리박람회 & 개강축제 행사 안내
-> 이미지 전용 공지 발견: [국제교류] 25-2 국제교류프로그램설명회
-> 이미지 전용 공지 발견: 2025년도 (재)광산장학회 장학생 선발 공고(9/12 마감)
-> 이미지 전용 공지 발견: 2025년 하반기 제23기 후기 삼원 장학생(시각디자인전공) 선발 안내(9/26 마감)
-> 이미지 전용 공지 발견: 대학생이 반드시 지켜야 할 저작권 상식
-> 이미지 전용 공지 발견: 2025년 충청남도평생교육인재육성진흥원 학자금 대출이자 지원사업 안내(9/26 마감)
총 60개의 공지사항이 성공적으로 저장되었습니다!
이미지만 있는 공지는 총 16개입니다.
